In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# Reflect an existing database into a new model
Base = automap_base()

# Reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:

# View all of the classes that automap found
Base.classes.keys()

In [ ]:

# View all of the classes that automap found
Base.classes.keys()

In [ ]:

# View all of the classes that automap found
Base.classes.keys()

In [ ]:

# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Grab last date from selected date range
last_date = session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).first()[0]

# Convert object type
entered_date = dt.datetime.strptime(last_date, '%Y-%m-%d')
entered_date = entered_date.date()

# Calculate the date 1 year ago from the last data point in the database
yr_ago = entered_date - dt.timedelta(days=365)
yr_ago

# Perform a query to retrieve the data and precipitation scores
one_year_prcp = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= yr_ago).\
    filter(Measurement.date <= last_date).all()

In [ ]:

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(one_year_prcp, columns=['date', 'prcp'])
df.set_index('date', inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data
df.plot()
plt.xticks([])
plt.savefig('Images/precipitation.png')
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# Query how many stations are available in this dataset
station_ct = session.query(Measurement.station).distinct().count()
print("Station Count:", station_ct)

In [ ]:
# List most active stations (stations with the most rows of data) in descending order.
active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()

active_stations_df = pd.DataFrame(active_stations, columns=['Station', 'Data Count'])
active_stations_df

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station
most_active_station = active_stations[0][0]

sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]

most_active_station_tobs = session.query(*sel).\
    filter(Measurement.station==most_active_station).all()[0]

print(f"Temperature Minimum: {most_active_station_tobs[0]}")
print(f"Temperature Average: {most_active_station_tobs[1]}")
print(f"Temperature Maximum: {most_active_station_tobs[2]}")

In [ ]:

# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
stn_tobs = session.query(Measurement.tobs).\
    filter(Measurement.date >= yr_ago).\
    filter(Measurement.date <= last_date).\
    filter(Measurement.station==most_active_station).all()

tobs_array = np.ravel(stn_tobs)
tobs_series = pd.Series(tobs_array)

tobs_series.plot.hist(bins=12)
plt.title("Temperature Observations of Station USC00519281")
plt.savefig('Images/station-histogram.png')
plt.show()

In [ ]:

import scipy.stats as stats
from random import sample

In [ ]:
# Query average temperatures for June
june_temps = session.query(Measurement.date, Measurement.station, func.avg(Measurement.tobs)).\
    filter(func.strftime("%m", Measurement.date) == "06").\
    group_by(Measurement.station, Measurement.date).all()

# Query average temperatures for December
dec_temps = session.query(Measurement.date, Measurement.station, func.avg(Measurement.tobs)).\
    filter(func.strftime("%m", Measurement.date) == "12").\
    group_by(Measurement.station, Measurement.date).all()

In [ ]:
# Store temperatures for each month in lists
june_temps = [i[2] for i in june_temps]
dec_temps = [i[2] for i in dec_temps]

# Check the length of samples
print(len(june_temps))
print(len(dec_temps))

In [ ]:

# Match the 'june_temps' sample size to 'dec_temps'
june_temps = sample(june_temps, len(dec_temps))

# Perform paired t-test and retrieve p-value
stats.ttest_rel(june_temps,dec_temps).pvalue

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

In [ ]:

# Use function `calc_temps` to calculate the tmin, tavg, and tmax 
# for trip dates using the previous year's data for those same dates.
temps = calc_temps('2017-06-10', '2017-06-25')[0]

print(f"Temperature Minimum: {temps[0]}")
print(f"Temperature Average: {temps[1]}")
print(f"Temperature Maxiumum: {temps[2]}")

In [ ]:
# Plot results from previous query as a bar chart with error bar. 

y = temps[1] # average temperature
yerr = temps[2] - temps[0] # peak-to-peak (tmax-tmin) value as y error

fig, ax = plt.subplots()
ax.bar(1, y, yerr=yerr, align='center', alpha=0.5, capsize=5)
ax.set_ylabel('Temperature (F)')
ax.set_title('Average Temperature in Between June 10-25, 2017')
ax.set_xlim(0,2)
ax.set_xticks([])
ax.set_ylim(0,100)
plt.tight_layout()
plt.savefig('Images/calc-temps.png')
plt.show()


In [ ]:
# Calculate the total amount of rainfall per weather station for trip dates using the previous year's matching dates.
# Sort in descending order by precipitation amount, listing the station, name, latitude, longitude, and elevation
start_dt = dt.datetime(2017, 6, 10)
end_dt = dt.datetime(2017, 6, 25)

sel = [Measurement.station, func.sum(Measurement.prcp), 
       Station.name, Station.latitude, Station.longitude, 
       Station.elevation]

trip_dates_results = session.query(*sel).\
    filter(Measurement.station == Station.station).\
    filter(Measurement.date>=start_dt).\
    filter(Measurement.date<=end_dt).\
    group_by(Measurement.station).\
    order_by(func.sum(Measurement.prcp).desc()).all()

trip_df = pd.DataFrame(trip_dates_results, columns=['station', 'prcp', 'name', 'latitude', 'longitude', 'elevation'])
trip_df

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)
# Example: daily_normals("01-01")

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

In [ ]:

# Calculate the daily normals for your trip push each tuple of calculations into a list called `normals`

# Use the start and end date from above to create a range of dates
trip_dates = pd.date_range(start_dt, end_dt)
trip_dates = trip_dates.astype(str)

# Strip off the year and save a list of %m-%d strings
split_year = [x.split("-", 1) for x in trip_dates]
date_str = [x[1] for x in split_year]
date_str

# Loop through the list of %m-%d strings and calculate the normals for each date
normals = [daily_normals(n) for n in date_str]

In [ ]:
# Extracts tuples into lists
tmin_list = []
tavg_list = []
tmax_list = []
dn_list = []

for n in normals: dn_list.append(n[0])

for tmin, tavg, tmax in dn_list:
    tmin_list.append(tmin)
    tavg_list.append(tavg)
    tmax_list.append(tmax)

In [ ]:
# Load query results into a Pandas DataFrame with trip dates as index
daily_norms_df = pd.DataFrame({"TMIN":tmin_list, "TAVG":tavg_list, "TMAX":tmax_list}, index=trip_dates)
daily_norms_df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
daily_norms_df.plot(kind='area', figsize=(8,6), stacked=False, alpha=0.25, rot=45)
plt.savefig('Images/daily-norms.png')
plt.tight_layout()
plt.show()